This script demonstrates basic use of catboost (CV 0.377750059602, LB 0.3802708)

I believe higher scores are possible - I got better early results without using aisle, so there may be an issue with how I set it up - maybe you can make it work better? ;)

The core of the script is based from "LB 0.3805009, Python Edition" by 🐳鲲(China), the catboost settings are taken from Fred Navruzov's posts, and the CV code is reused from my earlier notebook.

#### Original notes:

This script is translate from @Fabienvs's [R code](https://www.kaggle.com/fabienvs/instacart-xgboost-starter-lb-0-3791), I think it may help kagglers who do not use R.  
I really appreciate @Fabienvs's great work, to be honest, I have no idea about how to handling this kind of problem(this is the first time I encounter recommendation problem- -)  
here we go!! 
below exist some very useful functions I write by my own, you can download from my [github repo](https://github.com/NickYi1990/Kaggle_Buddy.git)
sorry for adding some Chinese in it, it would not affect the code
## The dataset is too big, you should run it on your desktop!

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 
import gc
import time
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

__MACOSX
aisles.csv
aisles.csv.zip
departments.csv
departments.csv.zip
order_products__prior.csv
order_products__prior.csv.zip
order_products__train.csv
order_products__train.csv.zip
orders.csv
orders.csv.zip
products.csv
products.csv.zip
sample_submission.csv
sample_submission.csv (1).zip



In [2]:
import xgboost
import catboost

/opt/conda/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module '_catboost' does not match runtime version 3.6
  return f(*args, **kwds)


In [3]:
def load_data(path_data):
    '''
    --------------------------------order_product--------------------------------
    * Unique in order_id + product_id
    '''
    priors = pd.read_csv(path_data + 'order_products__prior.csv', 
                     dtype={
                            'order_id': np.int32,
                            'product_id': np.uint16,
                            'add_to_cart_order': np.int16,
                            'reordered': np.int8})
    train = pd.read_csv(path_data + 'order_products__train.csv', 
                    dtype={
                            'order_id': np.int32,
                            'product_id': np.uint16,
                            'add_to_cart_order': np.int16,
                            'reordered': np.int8})
    '''
    --------------------------------order--------------------------------
    * This file tells us which set (prior, train, test) an order belongs
    * Unique in order_id
    * order_id in train, prior, test has no intersection
    * this is the #order_number order of this user
    '''
    orders = pd.read_csv(path_data + 'orders.csv', 
                         dtype={
                                'order_id': np.int32,
                                'user_id': np.int64,
                                'eval_set': 'category',
                                'order_number': np.int16,
                                'order_dow': np.int8,
                                'order_hour_of_day': np.int8,
                                'days_since_prior_order': np.float32})

    #  order in prior, train, test has no duplicate
    #  order_ids_pri = priors.order_id.unique()
    #  order_ids_trn = train.order_id.unique()
    #  order_ids_tst = orders[orders.eval_set == 'test']['order_id'].unique()
    #  print(set(order_ids_pri).intersection(set(order_ids_trn)))
    #  print(set(order_ids_pri).intersection(set(order_ids_tst)))
    #  print(set(order_ids_trn).intersection(set(order_ids_tst)))

    '''
    --------------------------------product--------------------------------
    * Unique in product_id
    '''
    products = pd.read_csv(path_data + 'products.csv')
    aisles = pd.read_csv(path_data + "aisles.csv")
    departments = pd.read_csv(path_data + "departments.csv")
    sample_submission = pd.read_csv(path_data + "sample_submission.csv")
    
    return priors, train, orders, products, aisles, departments, sample_submission

class tick_tock:
    def __init__(self, process_name, verbose=1):
        self.process_name = process_name
        self.verbose = verbose
    def __enter__(self):
        if self.verbose:
            print(self.process_name + " begin ......")
            self.begin_time = time.time()
    def __exit__(self, type, value, traceback):
        if self.verbose:
            end_time = time.time()
            print(self.process_name + " end ......")
            print('time lapsing {0} s \n'.format(end_time - self.begin_time))
            
def ka_add_groupby_features_1_vs_n(df, group_columns_list, agg_dict, only_new_feature=True):
    '''Create statistical columns, group by [N columns] and compute stats on [N column]

       Parameters
       ----------
       df: pandas dataframe
          Features matrix
       group_columns_list: list_like
          List of columns you want to group with, could be multiple columns
       agg_dict: python dictionary

       Return
       ------
       new pandas dataframe with original columns and new added columns

       Example
       -------
       {real_column_name: {your_specified_new_column_name : method}}
       agg_dict = {'user_id':{'prod_tot_cnts':'count'},
                   'reordered':{'reorder_tot_cnts_of_this_prod':'sum'},
                   'user_buy_product_times': {'prod_order_once':lambda x: sum(x==1),
                                              'prod_order_more_than_once':lambda x: sum(x==2)}}
       ka_add_stats_features_1_vs_n(train, ['product_id'], agg_dict)
    '''
    with tick_tock("add stats features"):
        try:
            if type(group_columns_list) == list:
                pass
            else:
                raise TypeError(k + "should be a list")
        except TypeError as e:
            print(e)
            raise

        df_new = df.copy()
        grouped = df_new.groupby(group_columns_list)

        the_stats = grouped.agg(agg_dict)
        the_stats.columns = the_stats.columns.droplevel(0)
        the_stats.reset_index(inplace=True)
        if only_new_feature:
            df_new = the_stats
        else:
            df_new = pd.merge(left=df_new, right=the_stats, on=group_columns_list, how='left')

    return df_new

def ka_add_groupby_features_n_vs_1(df, group_columns_list, target_columns_list, methods_list, keep_only_stats=True, verbose=1):
    '''Create statistical columns, group by [N columns] and compute stats on [1 column]

       Parameters
       ----------
       df: pandas dataframe
          Features matrix
       group_columns_list: list_like
          List of columns you want to group with, could be multiple columns
       target_columns_list: list_like
          column you want to compute stats, need to be a list with only one element
       methods_list: list_like
          methods that you want to use, all methods that supported by groupby in Pandas

       Return
       ------
       new pandas dataframe with original columns and new added columns

       Example
       -------
       ka_add_stats_features_n_vs_1(train, group_columns_list=['x0'], target_columns_list=['x10'])
    '''
    with tick_tock("add stats features", verbose):
        dicts = {"group_columns_list": group_columns_list , "target_columns_list": target_columns_list, "methods_list" :methods_list}

        for k, v in dicts.items():
            try:
                if type(v) == list:
                    pass
                else:
                    raise TypeError(k + "should be a list")
            except TypeError as e:
                print(e)
                raise

        grouped_name = ''.join(group_columns_list)
        target_name = ''.join(target_columns_list)
        combine_name = [[grouped_name] + [method_name] + [target_name] for method_name in methods_list]

        df_new = df.copy()
        grouped = df_new.groupby(group_columns_list)

        the_stats = grouped[target_name].agg(methods_list).reset_index()
        the_stats.columns = [grouped_name] + \
                            ['_%s_%s_by_%s' % (grouped_name, method_name, target_name) \
                             for (grouped_name, method_name, target_name) in combine_name]
        if keep_only_stats:
            return the_stats
        else:
            df_new = pd.merge(left=df_new, right=the_stats, on=group_columns_list, how='left')
        return df_new

In [4]:
path_data = '../input/'
priors, train, orders, products, aisles, departments, sample_submission = load_data(path_data)


### Time-saving measure for local work - cache a local copy of the final dataframe 

In [5]:
try:
    data = pd.read_pickle('kernel38-data.pkl')
except:

    # Product part

    # Products information ----------------------------------------------------------------
    # add order information to priors set
    priors_orders_detail = orders.merge(right=priors, how='inner', on='order_id')

    # create new variables
    # _user_buy_product_times: 用户是第几次购买该商品
    priors_orders_detail.loc[:,'_user_buy_product_times'] = priors_orders_detail.groupby(['user_id', 'product_id']).cumcount() + 1
    # _prod_tot_cnts: 该商品被购买的总次数,表明被喜欢的程度
    # _reorder_tot_cnts_of_this_prod: 这件商品被再次购买的总次数
    ### 我觉得下面两个很不好理解，考虑改变++++++++++++++++++++++++++
    # _prod_order_once: 该商品被购买一次的总次数
    # _prod_order_more_than_once: 该商品被购买一次以上的总次数
    agg_dict = {'user_id':{'_prod_tot_cnts':'count'}, 
                'reordered':{'_prod_reorder_tot_cnts':'sum'}, 
                '_user_buy_product_times': {'_prod_buy_first_time_total_cnt':lambda x: sum(x==1),
                                            '_prod_buy_second_time_total_cnt':lambda x: sum(x==2)}}
    prd = ka_add_groupby_features_1_vs_n(priors_orders_detail, ['product_id'], agg_dict)

    # _prod_reorder_prob: 这个指标不好理解
    # _prod_reorder_ratio: 商品复购率
    prd['_prod_reorder_prob'] = prd._prod_buy_second_time_total_cnt / prd._prod_buy_first_time_total_cnt
    prd['_prod_reorder_ratio'] = prd._prod_reorder_tot_cnts / prd._prod_tot_cnts
    prd['_prod_reorder_times'] = 1 + prd._prod_reorder_tot_cnts / prd._prod_buy_first_time_total_cnt

    prd.head()

    # User Part

    # _user_total_orders: 用户的总订单数
    # 可以考虑加入其它统计指标++++++++++++++++++++++++++
    # _user_sum_days_since_prior_order: 距离上次购买时间(和),这个只能在orders表里面计算，priors_orders_detail不是在order level上面unique
    # _user_mean_days_since_prior_order: 距离上次购买时间(均值)
    agg_dict_2 = {'order_number':{'_user_total_orders':'max'},
                  'days_since_prior_order':{'_user_sum_days_since_prior_order':'sum', 
                                            '_user_mean_days_since_prior_order': 'mean'}}
    users = ka_add_groupby_features_1_vs_n(orders[orders.eval_set == 'prior'], ['user_id'], agg_dict_2)

    # _user_reorder_ratio: reorder的总次数 / 第一单后买后的总次数
    # _user_total_products: 用户购买的总商品数
    # _user_distinct_products: 用户购买的unique商品数
    agg_dict_3 = {'reordered':
                  {'_user_reorder_ratio': 
                   lambda x: sum(priors_orders_detail.ix[x.index,'reordered']==1)/
                             sum(priors_orders_detail.ix[x.index,'order_number'] > 1)},
                  'product_id':{'_user_total_products':'count', 
                                '_user_distinct_products': lambda x: x.nunique()}}
    us = ka_add_groupby_features_1_vs_n(priors_orders_detail, ['user_id'], agg_dict_3)
    users = users.merge(us, how='inner')

    # 平均每单的商品数
    # 每单中最多的商品数，最少的商品数++++++++++++++
    users['_user_average_basket'] = users._user_total_products / users._user_total_orders

    us = orders[orders.eval_set != "prior"][['user_id', 'order_id', 'eval_set', 'days_since_prior_order']]
    us.rename(index=str, columns={'days_since_prior_order': 'time_since_last_order'}, inplace=True)

    users = users.merge(us, how='inner')

    users.head()

    # Database Part

    # 这里应该还有很多变量可以被添加
    # _up_order_count: 用户购买该商品的次数
    # _up_first_order_number: 用户第一次购买该商品所处的订单数
    # _up_last_order_number: 用户最后一次购买该商品所处的订单数
    # _up_average_cart_position: 该商品被添加到购物篮中的平均位置
    agg_dict_4 = {'order_number':{'_up_order_count': 'count', 
                                  '_up_first_order_number': 'min', 
                                  '_up_last_order_number':'max'}, 
                  'add_to_cart_order':{'_up_average_cart_position': 'mean'}}

    data = ka_add_groupby_features_1_vs_n(df=priors_orders_detail, 
                                                          group_columns_list=['user_id', 'product_id'], 
                                                          agg_dict=agg_dict_4)

    data = data.merge(prd, how='inner', on='product_id').merge(users, how='inner', on='user_id')
    # 该商品购买次数 / 总的订单数
    # 最近一次购买商品 - 最后一次购买该商品
    # 该商品购买次数 / 第一次购买该商品到最后一次购买商品的的订单数
    data['_up_order_rate'] = data._up_order_count / data._user_total_orders
    data['_up_order_since_last_order'] = data._user_total_orders - data._up_last_order_number
    data['_up_order_rate_since_first_order'] = data._up_order_count / (data._user_total_orders - data._up_first_order_number + 1)

    # add user_id to train set
    train = train.merge(right=orders[['order_id', 'user_id']], how='left', on='order_id')
    data = data.merge(train[['user_id', 'product_id', 'reordered']], on=['user_id', 'product_id'], how='left')

    # release Memory
    # del train, prd, users
    # gc.collect()
    # release Memory
    del priors_orders_detail, orders
    gc.collect()

    data.head()
    
    data.to_pickle('kernel38-data.pkl')

## Validation code

In [6]:
try:
    df_train_gt = pd.read_csv('train.csv', index_col='order_id')
except:
    train_gtl = []

    for uid, subset in train_details.groupby('user_id'):
        subset1 = subset[subset.reordered == 1]
        oid = subset.order_id.values[0]

        if len(subset1) == 0:
            train_gtl.append((oid, 'None'))
            continue

        ostr = ' '.join([str(int(e)) for e in subset1.product_id.values])
        # .strip is needed because join can have a padding space at the end
        train_gtl.append((oid, ostr.strip()))

    df_train_gt = pd.DataFrame(train_gtl)

    df_train_gt.columns = ['order_id', 'products']
    df_train_gt.set_index('order_id', inplace=True)
    df_train_gt.sort_index(inplace=True)
    
    df_train_gt.to_csv('train.csv')

In [7]:
def compare_results(df_gt, df_preds):
    
    df_gt_cut = df_gt.loc[df_preds.index]
    
    f1 = []
    for gt, pred in zip(df_gt_cut.sort_index().products, df_preds.sort_index().products):
        lgt = gt.replace("None", "-1").split(' ')
        lpred = pred.replace("None", "-1").split(' ')

        rr = (np.intersect1d(lgt, lpred))
        precision = np.float(len(rr)) / len(lpred)
        recall = np.float(len(rr)) / len(lgt)

        denom = precision + recall
        f1.append(((2 * precision * recall) / denom) if denom > 0 else 0)

    #print(np.mean(f1))
    return(np.mean(f1))

# Create Train / Test

### Add aisle and dept id for catboost

In [8]:
data = pd.merge(data, products.drop('product_name', axis=1), on='product_id')

In [16]:
X_test = data.loc[data.eval_set == "test",:].copy()
X_test.drop(['eval_set'], axis=1, inplace=True)

In [9]:
train = data.loc[data.eval_set == "train",:].copy()
train.drop(['eval_set'], axis=1, inplace=True)
train.loc[:, 'reordered'] = train.reordered.fillna(0)

### perform CV for catboost 

In [10]:
def catboost_cv(X_train, y_train, X_val, y_val, features_to_use):
    cb = catboost.CatBoostClassifier(iterations=100, 
        thread_count=8, 
        verbose=True,
        random_seed=42,
        learning_rate=0.05,
        use_best_model=True,
        depth=8,
        fold_permutation_block_size=64,
        calc_feature_importance=True,
        leaf_estimation_method='Gradient')

    cb.fit(X=X_train[features_to_use].values, 
            y=y_train.values, 
            #cat_features=[c.get_loc('aisle_id'), c.get_loc('department_id')], 
            cat_features=[X_train[features_to_use].columns.get_loc('aisle_id')], 
            eval_set=[X_val[features_to_use].values, y_val.values],
            verbose=True,
            #plot=True
           )
    
    return cb

In [ ]:
df_cvfolds = []
cb = []

for fold in range(4):
    train_subset = train[train.user_id % 4 != fold]
    valid_subset = train[train.user_id % 4 == fold]

    X_train = train_subset.drop('reordered', axis=1)
    y_train = train_subset.reordered

    X_val = valid_subset.drop('reordered', axis=1)
    y_val = valid_subset.reordered

    val_index = X_val[['user_id', 'product_id', 'order_id']]
    
    features_to_use = list(X_train.columns)
    features_to_use.remove('user_id')
    features_to_use.remove('product_id')
    features_to_use.remove('order_id')

    cb.append(catboost_cv(X_train, y_train, X_val, y_val, features_to_use))
    rawpreds = cb[-1].predict_proba(X_val[features_to_use].values)

    lim = .202
    val_out = val_index.copy()

    val_out.loc[:,'reordered'] = (rawpreds[:,1] > lim).astype(int)
    val_out.loc[:, 'product_id'] = val_out.product_id.astype(str)
    presubmit = ka_add_groupby_features_n_vs_1(val_out[val_out.reordered == 1], 
                                                   group_columns_list=['order_id'],
                                                   target_columns_list= ['product_id'],
                                                   methods_list=[lambda x: ' '.join(set(x))], keep_only_stats=True)

    presubmit = presubmit.set_index('order_id')
    presubmit.columns = ['products']

    fullfold = pd.DataFrame(index = val_out.order_id.unique())

    fullfold.index.name = 'order_id'
    fullfold['products'] = ['None'] * len(fullfold)

    fullfold.loc[presubmit.index, 'products'] = presubmit.products

    print(fold, compare_results(df_train_gt, fullfold))
    
    df_cvfolds.append(fullfold)

Borders generated
0:	learn 0.635726643	test 0.6356663916	bestTest 0.6356663916	passed: 24.1 sec	total: 35.4s	remaining: 58m 26s
1:	learn 0.5849748625	test 0.5848673064	bestTest 0.5848673064	passed: 24 sec	total: 59.4s	remaining: 48m 31s
2:	learn 0.5419628413	test 0.5418079236	bestTest 0.5418079236	passed: 24.1 sec	total: 1m 23s	remaining: 44m 59s
3:	learn 0.5043970358	test 0.5041678733	bestTest 0.5041678733	passed: 24.2 sec	total: 1m 47s	remaining: 43m 3s
4:	learn 0.471797413	test 0.4715476458	bestTest 0.4715476458	passed: 23.3 sec	total: 2m 10s	remaining: 41m 28s
5:	learn 0.4438987925	test 0.4435939042	bestTest 0.4435939042	passed: 23.2 sec	total: 2m 34s	remaining: 40m 15s
6:	learn 0.419668827	test 0.4193259158	bestTest 0.4193259158	passed: 21.9 sec	total: 2m 56s	remaining: 38m 58s
7:	learn 0.3989645142	test 0.3986051661	bestTest 0.3986051661	passed: 22.3 sec	total: 3m 18s	remaining: 38m
8:	learn 0.3816517539	test 0.3812821972	bestTest 0.3812821972	passed: 23.7 sec	total: 3m 42s	remai

73:	learn 0.247493069	test 0.2468665488	bestTest 0.2468665488	passed: 24.8 sec	total: 28m 59s	remaining: 10m 11s
74:	learn 0.2474245037	test 0.2467971193	bestTest 0.2467971193	passed: 24.1 sec	total: 29m 23s	remaining: 9m 47s
75:	learn 0.2473777873	test 0.2467520638	bestTest 0.2467520638	passed: 23.8 sec	total: 29m 47s	remaining: 9m 24s
76:	learn 0.2473111497	test 0.2466871984	bestTest 0.2466871984	passed: 23.6 sec	total: 30m 11s	remaining: 9m 1s
77:	learn 0.2472480239	test 0.2466231591	bestTest 0.2466231591	passed: 23.8 sec	total: 30m 34s	remaining: 8m 37s
78:	learn 0.2472141657	test 0.2465896941	bestTest 0.2465896941	passed: 24.6 sec	total: 30m 59s	remaining: 8m 14s
79:	learn 0.2471640091	test 0.2465413823	bestTest 0.2465413823	passed: 24.7 sec	total: 31m 24s	remaining: 7m 51s
80:	learn 0.247124541	test 0.2465062175	bestTest 0.2465062175	passed: 23.1 sec	total: 31m 47s	remaining: 7m 27s
81:	learn 0.2470711883	test 0.2464530508	bestTest 0.2464530508	passed: 24.2 sec	total: 32m 11s	rem

44:	learn 0.2515484138	test 0.2525914023	bestTest 0.2525914023	passed: 21.7 sec	total: 17m 53s	remaining: 21m 51s
45:	learn 0.2511832685	test 0.2522255934	bestTest 0.2522255934	passed: 23.1 sec	total: 18m 16s	remaining: 21m 26s
46:	learn 0.2508777231	test 0.2519211029	bestTest 0.2519211029	passed: 21.5 sec	total: 18m 37s	remaining: 21m
47:	learn 0.2505530344	test 0.2515995629	bestTest 0.2515995629	passed: 23.3 sec	total: 19m	remaining: 20m 36s
48:	learn 0.2502603046	test 0.251309548	bestTest 0.251309548	passed: 22.2 sec	total: 19m 23s	remaining: 20m 10s
49:	learn 0.2500240016	test 0.2510774176	bestTest 0.2510774176	passed: 23.8 sec	total: 19m 46s	remaining: 19m 46s
50:	learn 0.2497736772	test 0.250827937	bestTest 0.250827937	passed: 22.2 sec	total: 20m 9s	remaining: 19m 21s
51:	learn 0.2495593206	test 0.2506127093	bestTest 0.2506127093	passed: 24.4 sec	total: 20m 33s	remaining: 18m 58s
52:	learn 0.2493635276	test 0.2504199291	bestTest 0.2504199291	passed: 23 sec	total: 20m 56s	remainin

15:	learn 0.3068149616	test 0.3064833807	bestTest 0.3064833807	passed: 22.8 sec	total: 6m 23s	remaining: 33m 33s
16:	learn 0.3004465282	test 0.3001253125	bestTest 0.3001253125	passed: 21.9 sec	total: 6m 45s	remaining: 32m 59s
17:	learn 0.2949060151	test 0.2945758338	bestTest 0.2945758338	passed: 23.4 sec	total: 7m 8s	remaining: 32m 34s
18:	learn 0.290316758	test 0.2899885344	bestTest 0.2899885344	passed: 23.6 sec	total: 7m 32s	remaining: 32m 9s
19:	learn 0.2860926377	test 0.2857658792	bestTest 0.2857658792	passed: 24.2 sec	total: 7m 56s	remaining: 31m 46s
20:	learn 0.2824421162	test 0.2821228443	bestTest 0.2821228443	passed: 23.5 sec	total: 8m 20s	remaining: 31m 21s
21:	learn 0.2792367717	test 0.2789215289	bestTest 0.2789215289	passed: 23.2 sec	total: 8m 43s	remaining: 30m 55s
22:	learn 0.2761069567	test 0.275792049	bestTest 0.275792049	passed: 21.9 sec	total: 9m 5s	remaining: 30m 25s
23:	learn 0.2736204892	test 0.2733093631	bestTest 0.2733093631	passed: 23.8 sec	total: 9m 29s	remainin

88:	learn 0.2466891732	test 0.2465382032	bestTest 0.2465382032	passed: 22.6 sec	total: 34m 46s	remaining: 4m 17s
89:	learn 0.2466582498	test 0.2465094669	bestTest 0.2465094669	passed: 23.5 sec	total: 35m 9s	remaining: 3m 54s
90:	learn 0.2466295303	test 0.2464831284	bestTest 0.2464831284	passed: 24 sec	total: 35m 34s	remaining: 3m 31s
91:	learn 0.2465940427	test 0.2464485141	bestTest 0.2464485141	passed: 22.9 sec	total: 35m 56s	remaining: 3m 7s
92:	learn 0.246570825	test 0.2464268036	bestTest 0.2464268036	passed: 23.9 sec	total: 36m 20s	remaining: 2m 44s
93:	learn 0.24654499	test 0.2464045945	bestTest 0.2464045945	passed: 23.7 sec	total: 36m 44s	remaining: 2m 20s
94:	learn 0.2465157923	test 0.2463833923	bestTest 0.2463833923	passed: 22.4 sec	total: 37m 6s	remaining: 1m 57s
95:	learn 0.2464906093	test 0.2463614805	bestTest 0.2463614805	passed: 24.1 sec	total: 37m 31s	remaining: 1m 33s
96:	learn 0.2464661791	test 0.2463418441	bestTest 0.2463418441	passed: 25.2 sec	total: 37m 56s	remaining

59:	learn 0.2484527697	test 0.2485937282	bestTest 0.2485937282	passed: 22.2 sec	total: 23m 4s	remaining: 15m 23s
60:	learn 0.2483270821	test 0.2484688775	bestTest 0.2484688775	passed: 23 sec	total: 23m 27s	remaining: 14m 59s
61:	learn 0.2482133573	test 0.2483562713	bestTest 0.2483562713	passed: 24.2 sec	total: 23m 51s	remaining: 14m 37s
62:	learn 0.2481166029	test 0.2482630664	bestTest 0.2482630664	passed: 23 sec	total: 24m 14s	remaining: 14m 14s
63:	learn 0.248028341	test 0.2481776791	bestTest 0.2481776791	passed: 22.4 sec	total: 24m 37s	remaining: 13m 50s
64:	learn 0.2479457058	test 0.2480987823	bestTest 0.2480987823	passed: 23.8 sec	total: 25m 1s	remaining: 13m 28s
65:	learn 0.2478596601	test 0.248014643	bestTest 0.248014643	passed: 22.1 sec	total: 25m 23s	remaining: 13m 4s
66:	learn 0.2477567326	test 0.2479136171	bestTest 0.2479136171	passed: 23.5 sec	total: 25m 46s	remaining: 12m 41s
67:	learn 0.247679815	test 0.2478407914	bestTest 0.2478407914	passed: 23 sec	total: 26m 9s	remaini

In [15]:
df_cv = pd.concat(df_cvfolds)
print(compare_results(df_train_gt, df_cv))

0.377750059602


In [42]:
### now run model with 100% of train set for submission

In [18]:
cb_full = catboost.CatBoostClassifier(iterations=100, 
    thread_count=8, 
    verbose=True,
    random_seed=42,
    learning_rate=0.05,
    depth=8,
    fold_permutation_block_size=64,
    calc_feature_importance=True,
    leaf_estimation_method='Gradient')

cb_full.fit(X=train[features_to_use].values, 
        y=train.reordered.values, 
        cat_features=[train[features_to_use].columns.get_loc('aisle_id')], 
        verbose=True,
        #plot=True
       )

Borders generated
0:	learn 0.6349939378passed: 32 sec	total: 44.3s	remaining: 1h 13m 5s
1:	learn 0.5848126827passed: 31.1 sec	total: 1m 15s	remaining: 1h 1m 36s
2:	learn 0.5413888139passed: 31.5 sec	total: 1m 46s	remaining: 57m 37s
3:	learn 0.5043940462passed: 33 sec	total: 2m 19s	remaining: 55m 59s
4:	learn 0.472509014passed: 33.3 sec	total: 2m 53s	remaining: 54m 52s
5:	learn 0.445319348passed: 32.5 sec	total: 3m 25s	remaining: 53m 43s
6:	learn 0.4209626024passed: 32.8 sec	total: 3m 58s	remaining: 52m 48s
7:	learn 0.3999683915passed: 31.2 sec	total: 4m 29s	remaining: 51m 41s
8:	learn 0.3820880466passed: 30.7 sec	total: 5m	remaining: 50m 37s
9:	learn 0.3664413051passed: 30.2 sec	total: 5m 30s	remaining: 49m 35s
10:	learn 0.3527427143passed: 32.7 sec	total: 6m 3s	remaining: 48m 59s
11:	learn 0.3409674792passed: 32.5 sec	total: 6m 35s	remaining: 48m 22s
12:	learn 0.3303723961passed: 33.8 sec	total: 7m 9s	remaining: 47m 54s
13:	learn 0.3213983543passed: 31.3 sec	total: 7m 40s	remaining: 4

### Prepare submission

In [20]:
testpreds = X_test[['user_id', 'product_id', 'order_id']].copy()
testpreds['reordered'] = (cb_full.predict_proba(X_test[features_to_use].values)[:,1] > .202).astype(int)
testpreds.product_id = testpreds.product_id.astype(str)

In [30]:
g = testpreds[testpreds.reordered == 1].groupby('order_id', sort=False)
df_testpreds = g[['product_id']].agg(lambda x: ' '.join(set(x)))

df_testpreds.head()

,product_id
order_id,
2161313,10441 14715 12427 37710 11266 196 27839
1735923,12108 34690 31487 17008 2192 35123 196 15131
139487,21386 196 6184
1797124,17957 22802 196 15541
1294240,46803 13176 16797 196 26900 32478


In [34]:
# complete (but empty) test df
df_test = pd.DataFrame(index=X_test.order_id.unique())
df_test.index.name = 'order_id'
df_test['products'] = ['None'] * len(df_test)

In [35]:
# yup, there are ~3345 order_id's with no orders
len(df_test), len(df_testpreds)

(75000, 71655)

In [37]:
# combine empty output df with predictions
df_test.loc[df_testpreds.index, 'products'] = df_testpreds.product_id
df_test.sort_index(inplace=True)
df_test.to_csv('preds_catboost.csv')